In [1]:
import re
import pandas as pd

In [2]:
from allegro import *

In [3]:
table = pd.read_html('https://www.cpubenchmark.net/cpu_list.php')

In [4]:
class Processor:
    
    def __init__(self, string):
        self.string = string
        self.price = float(string['sellingMode']['price']['amount'])
        self.delivery_price = 0 if string['delivery']['availableForFree'] else float(string['delivery']['lowestPrice']['amount'])
        self.total_price = self.price + self.delivery_price
        link = 'https://allegro.pl/oferta/' + string['id']
        self.link = link
        self.name = None


In [5]:
single_cpu, multi_cpu = table
single_cpu = single_cpu.rename({'CPU Name': 'name', 'CPU Mark(higher is better)': 'score'}, axis='columns', errors='raise')
single_cpu = single_cpu[['name', 'score']]

In [6]:
minBenchmark = 4000
used_sockets = ['1151', '1150', 'AM3', '1155', '1156', 'AM3+']
price_range = (0,150)
# k = loadDictionary('dict.txt', used_sockets)

auction_list = get_auction_data(sockets=used_sockets, price=price_range, mobo=False)

Loading page 1 out of 14
Loading page 2 out of 14
Loading page 3 out of 14
Loading page 4 out of 14
Loading page 5 out of 14
Loading page 6 out of 14
Loading page 7 out of 14
Loading page 8 out of 14
Loading page 9 out of 14
Loading page 10 out of 14
Loading page 11 out of 14
Loading page 12 out of 14
Loading page 13 out of 14
Loading page 14 out of 14


In [7]:
iseries = re.compile('i[3579]\s*-*\s*\d{3,4}[pktcsbehr(hq)(te)(eq)(kf)]*', re.I)
pentium = re.compile('g\s*\d{3,4}\s*[(te)t]*', re.I)
xeons   = re.compile('[elx]3*\s*-*\s*\d{4}[lg]*', re.I)
athlonx = re.compile('x[234p6]\s\d{3,4}[etu\+]*', re.I)
athlon = re.compile('(?<!fx-)(?<!fx)(?<!\w)(?<!fx\s)(?<!x[234p6]\s)[(le)(gp)(be)]*-*\d{4}[\+eb]*(?!\w)', re.I)
fxs = re.compile('fx\s*-*\s*b*\d{4}e*', re.I)
xnbs = re.compile('x[234]\sb\d{2}e*', re.I)

procs = [iseries,fxs, athlonx, athlon, xnbs]
#procs = [iseries, pentium, xeons]

In [8]:
df = pd.DataFrame(columns = ['name', 'price', 'total_price', 'link'])

In [9]:
for element in auction_list:
    cpu = Processor(element)
    
#     link_CPU = lambda name: 'https://www.cpubenchmark.net/cpu.php?cpu=' + re.sub('@', '%40', re.sub(' ', '+', name))

    for pattern in procs:

        if pattern.search(element['name'], re.I):

            #if motherboards then probably CPU name will be the last identified "G4560"-type key in the name
            if False: #is motherboard 'id=4228&' in URI:
                newName = pattern.findall(element['name'], re.I)[-1]
            else:
                newName = pattern.findall(element['name'], re.I)[0]

            #print('{0:50.50}: \033[0;32m {1:10.10} {2:10.10} {3:10.10} {4:10.10}\x1b[0m'.format(element['name'], newName, re.sub( '(?<=i[3579])\s(?=\d{3,4})', '-', newName, flags=re.I), re.sub( '(?<=fx)\s(?=\d{3,4})', '-', newName, flags=re.I), re.sub( '(?<![fxi]\w)[\s-]', '', newName, flags=re.I) ))
            #remove whitespaces and insert hyphen for i[357] and fx series   

            newName = re.sub( '(?<=i[3579])\s(?=\d{3,4})', '-', newName, flags=re.I)
#             print(newName)
            newName = re.sub( '(?<=fx)\s*(?=\d{3,4})', '-', newName, flags=re.I)
#             print(newName)
            #newName = re.sub( '\s', '', newName)
            newName = re.sub( '(?<![fxi]\w)[\s-]', '', newName, flags=re.I)
#             print(newName)
            break
        else:
            newName = 'Error'
    cpu.name = newName
    df = df.append({
        'name': cpu.name,
        'price': cpu.price,
        'total_price': cpu.total_price,
        'link': cpu.link}, ignore_index=True)

In [10]:
df['name'][0]

'I5-3570'

In [11]:
df

,name,price,total_price,link
0,I5-3570,149.00,157.99,https://allegro.pl/oferta/9176656887
1,i5-3470,114.00,122.99,https://allegro.pl/oferta/8405286122
2,x4 955,150.00,158.99,https://allegro.pl/oferta/9333749874
3,i3-4130,90.00,98.99,https://allegro.pl/oferta/9039330742
4,i5-3470,132.00,140.99,https://allegro.pl/oferta/9039143057
...,...,...,...,...
835,X3 445,55.00,63.99,https://allegro.pl/oferta/8596029159
836,i5-2500,129.99,138.98,https://allegro.pl/oferta/9263920094
837,I3-2120,119.00,124.90,https://allegro.pl/oferta/7993680114
838,I3-2120,29.99,49.99,https://allegro.pl/oferta/8987445079


In [96]:
cpu.string['name']

'Procesor intel core i5-3470 gwarancja Lga1155'

In [60]:
cpu.name

'3470'

In [169]:
single_cpu[single_cpu['name'].str.contains(cpu.name)]

,name,score
284,AMD Athlon II X3 445,1697


In [167]:
x

284    1697
Name: score, dtype: int64

In [159]:
single_cpu

,name,score
0,AMD A10 Micro-6700T APU,1257
1,AMD A10 PRO-7350B APU,1746
2,AMD A10 PRO-7800B APU,2983
3,AMD A10 PRO-7850B APU,3308
4,AMD A10-4600M APU,1896
...,...,...
2883,VIA QuadCore C4650@2.0GHz,1323
2884,VIA QuadCore L4700 @ 1.2+ GHz,740
2885,VIA QuadCore U4650 @ 1.0+ GHz,637
2886,ZHAOXIN KaiXian KX-U6780A@2.7GHz,5761


,name,score
0,AMD A10 Micro-6700T APU,1257
1,AMD A10 PRO-7350B APU,1746
2,AMD A10 PRO-7800B APU,2983
3,AMD A10 PRO-7850B APU,3308
4,AMD A10-4600M APU,1896
...,...,...
2883,VIA QuadCore C4650@2.0GHz,1323
2884,VIA QuadCore L4700 @ 1.2+ GHz,740
2885,VIA QuadCore U4650 @ 1.0+ GHz,637
2886,ZHAOXIN KaiXian KX-U6780A@2.7GHz,5761


In [77]:
cpu.string['name']

'Procesor intel core i5-3470 gwarancja Lga1155'

In [78]:
iseries = re.compile('i[3579][\s-]\d{3,4}[pktcsbehr(hq)(te)(eq)(kf)]*', re.I)

In [79]:
iseries.findall(cpu.string['name'])

['i5-3470']